# Date Mining

## Setup

In [296]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import yaml
import regex 

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# pd.set_option('display.max_rows', None)

sns.set_style("darkgrid")

from IPython.display import display, Markdown
from pprint import pprint 

DEBUG = True
SEED = 666

In [297]:
DATASET = "dates.csv"

import os, sys
COLAB = 'google.colab' in sys.modules
ROOT = "./"

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)


def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','output']: makedirs(d)

# Set the size of the columns in order to see all of the data
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Dataset

Downloads the dataset if it does not exist locally

In [298]:
if os.path.isfile(ROOT+ "orig/"+DATASET):
    print("Using local copy...")
    df = pd.read_csv(ROOT+ "orig/"+DATASET)
else:
    print("Downloading dataset...")
    df = pd.read_csv("https://setu-datamining2.github.io/live/topics/21-Assignments/03-Mining_Dates/files/public.csv")
    df.to_csv(ROOT+ "orig/"+DATASET, index=False)

Using local copy...


In [299]:
def info(df, regular_expressions, regex_index = -1):
    temp = df[df["RegexMatched"] == regex_index].copy()
    reg = regular_expressions[regex_index]["regex"] if regex_index >= 0 else "None"
    print(f"Regex: {reg}")
    print(f"Shape: {temp.shape}")
    temp = temp.sample(frac=1).reset_index(drop=True)
    display(temp.head(50))
    return temp

## Import / Clean

Create a list of the months of the year

In [300]:
months = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]

- Create a map of possible spelling mistakes for each month.
- There is also some code to generate each month's name without each of its letters. 
  - nb: we do not use `May` as it does not have any spelling mistakes in the dataset and generating spelling mistakes for it messes with `March` and causes issues.

In [301]:
# Replace all the month abbreviations/spelling mistakes with the full month name
months_map = {
    "jan": ["january", "janaury"],
    "feb": ["february"],
    "mar": ["march"],
    "may": ["may"],
    "apr": ["april"],
    "jun": ["june"],
    "jul": ["july"],
    "aug": ["august"],
    "sep": ["september"],
    "oct": ["october"],
    "nov": ["november"],
    "dec": ["december", "decemeber"],
}

for k, v in months_map.items():
    # Get the full month name from the list
    full_month_name = v[0]  

    if full_month_name == "may":
        continue

    # Generate 
    spelling_mistakes = []  
    for i in range(len(full_month_name)):
        spelling_mistakes.append(full_month_name[:i] + full_month_name[i+1:])

    v.extend(spelling_mistakes)

Cleaning Algorithm:
- Make the text lowercase
- Strip the ends and beginning of the text of any whitespace
- Replace any full month names and their spelling mistakes with the correct month abbreviation
- Remove some strings that are known to break up dates in the dataset.
- Remove all double spaces

In [302]:
def clean(raw_str):
    # Lowercase the string and remove leading/trailing whitespaces
    raw_str = raw_str.lower().strip()

    # Replace all the month names/spelling mistakes with the month abbreviation
    for k,v in months_map.items():
        for vv in v:
            if vv in raw_str:
                raw_str = raw_str.replace(vv, k)
                break # Break after the first match

    # Remove some words that could be breaking up the date
    to_remove = ["mo", ",", ".", "in"]

    for tr in to_remove:
        raw_str = raw_str.replace(tr," ")

    # Remove double spaces
    while "  " in raw_str:
        raw_str = raw_str.replace("  "," ")
    
    return raw_str

### Setup new columns

Apply the cleaning algorithm to the `Raw` column and create new columns for each month of the year.

In [303]:
df["Data"] = df.Raw.apply(clean)
df["RegexMatched"] = -1
df[["Day", "Month", "Year"]] = 0

## Applying Regular Expressions

This list of regular expressions is used to extract dates from the dataset. It will be run sequentially on the `Data` column.

In [304]:
months_regex = "|".join(months)

regular_expressions = [
    {
        "example": "12 sep 2024",
        "formats": ["mm mmm yyyy", "mm mmm yy"],
        "regex": r"(\d{1,2}) (%s) (\d{2,4})" % months_regex,
        "columns": ["day", "month", "year"]
    },
    {
        "example": "may 24 2009",
        "formats": ["mmm dd yy", "mmm dd yyyy"],
        "regex": r"(%s) (\d{1,2}) (\d{2,4})" % months_regex,
        "columns": ["month", "day", "year"]
    },
    {
        "example": "sep 2024",
        "formats": ["mmm yyyy", "mmm yy"],
        "regex": r"(%s) (\d{2,4})" % months_regex,
        "columns": ["month", "year"]
    },
    {
        "example": "12/31/2024",
        "formats": ["mm/dd/yyyy", "mm-dd-yyyy", "mm dd yyyy", "mm/dd/yy", "mm-dd-yy", "mm dd yy"],
        "regex": r"(\d{1,2})[-/](\d{1,2})[-/](\d{2,4})",
        "columns": ["month", "day", "year"]
    },
    {
        "example": "12/2024",
        "formats": ["mm/yyyy", "mm-yyyy", "mm yyyy", "mm/yy", "mm-yy", "mm yy"],
        "regex": r"(\d{1,2})[-/](\d{2,4})",
        "columns": ["month", "year"]
    },
    {
        "example": "2024",
        "formats": ["yyyy", "yy"],
        "regex": r"(\d{4})",
        "columns": ["year"]
    }
]


Check regular expression algorithm:
- Get all rows that have not been matched by any regular expression
- Extract the regular expression provided to the remaining rows
- Extract and update the `day`, `month`, and `year` from the regular expression to the `Day`, `Month`, and `Year` columns respectively

In [305]:
def check_regex(df, rgx):
    # Display the regex
    display(Markdown(f"### [{regular_expressions.index(rgx)}] Checking regex: `{rgx['regex']}`"))
 
    # Check the regex against the data that hasn't been matched yet
    df_to_check = df[df.RegexMatched == -1]
    matches = df_to_check.Data.str.extract(rgx["regex"])
    matches.dropna(inplace=True)
    matches.columns = rgx["columns"]

    # Update the original dataframe
    df.loc[matches.index, "RegexMatched"] = regular_expressions.index(rgx)

    if "day" in matches.columns:
        df.loc[matches.index, "Day"] = matches["day"].astype(int)

    if "month" in matches.columns:
        matches["month"] = matches["month"].apply(lambda x: months.index(x)+1 if x in months else x)
        df.loc[matches.index, "Month"] = matches["month"].astype(int)

    if "year" in matches.columns:
        df.loc[matches.index, "Year"] = matches["year"].astype(int)

    # Display the matches
    display(Markdown(f"#### Found {len(matches)} matches"))

    return df

In [306]:
for r in regular_expressions:
    df = check_regex(df, r)

### [0] Checking regex: `(\d{1,2}) (jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) (\d{2,4})`

#### Found 189 matches

### [1] Checking regex: `(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) (\d{1,2}) (\d{2,4})`

#### Found 159 matches

### [2] Checking regex: `(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec) (\d{2,4})`

...

#### Found 124 matches

### [4] Checking regex: `(\d{1,2})[-/](\d{2,4})`

#### Found 106 matches

### [5] Checking regex: `(\d{4})`

#### Found 38 matches

### Check for unparsed rows

Check if there are any rows that have not been parsed by any regular expression.

In [307]:
tmp = info(df, regular_expressions, regex_index=-1)
if tmp.shape[0] == 0:
    print("All dates have been matched!")

Regex: None
Shape: (0, 7)


,Code,Raw,Data,RegexMatched,Day,Month,Year


All dates have been matched!


## Build the `Date` column

- Default the `Day`, and `Month` columns to `01` and `01` respectively.
- If the `Year` column is 2 digits update it to be between `1924` and `2024`.
- Create a new column `Date` that is a combination of the `Day`, `Month`, and `Year` columns.

In [308]:
df["Day"] = df["Day"].replace(0, 1)
df["Month"] = df["Month"].replace(0, 1)
df["Year"] = df["Year"].apply(lambda x: x if x > 1924 else x + 2000 if x < 24 else x + 1900)
df["Date"] = pd.to_datetime(df[["Year", "Month", "Day"]], errors="coerce")

## Check for impossible dates

Check if there are any dates that are impossible. ie `Month` is greater than `12`, `Day` is greater than `31`, or `Year` is less than `1924` or greater than `2024`.

In [309]:
impossible_dates = df[(df.Day > 31) | (df.Month > 12) | (df.Year > 2024) | (df.Year < 1924) | (df.Date.isnull())]
print(f"Impossible dates: {impossible_dates.shape[0]}")
display(impossible_dates)

Impossible dates: 0


,Code,Raw,Data,RegexMatched,Day,Month,Year,Date


## Sort by Date

Use the sorting code provided to sort the dataset primarily by the `Date` column and secondarily by the `Raw` column.

In [310]:
df.sort_values(by=['Date','Raw'], inplace=True)

Check if the dataset is sorted correctly.

In [311]:
n = 5
print(f"First {n} rows:")
display(df.head(n))
print(f"Last {n} rows:")
display(df.tail(n))

First 5 rows:


,Code,Raw,Data,RegexMatched,Day,Month,Year,Date
340,5262388306,"(June 13 70) Crt-1.96, BUN-26; AST/ALT-16/22; WBC_12.6Activities of Daily Living (ADL) Bathing: Independent",(jun 13 70) crt-1 96 bun-26; ast/alt-16/22; wbc_12 6activities of daily liv g (adl) bath g: dependent,1,13,6,1970,1970-06-13
7,1082469285,"Septeber, 10, 70 CPT Code: 90792: With medical services",sep 10 70 cpt code: 90792: with medical services,1,10,9,1970,1970-09-10
595,8335477666,"October, 21 70 Communication with referring physician?: Done",oct 21 70 communication with referr g physician?: done,1,21,10,1970,1970-10-21
443,6574729211,"Noember 16, 70 Total time of visit (in minutes):",nov 16 70 total time of visit ( m utes):,1,16,11,1970,1970-11-16
325,5043827109,"23, Deember 70 Age:",23 dec 70 age:,0,23,12,1970,1970-12-23


Last 5 rows:


,Code,Raw,Data,RegexMatched,Day,Month,Year,Date
502,7241000614,Sptember 13 17 Age:,sep 13 17 age:,1,13,9,2017,2017-09-13
475,6887830410,"30 September, 17 CPT Code: 90792: With medical services",30 sep 17 cpt code: 90792: with medical services,0,30,9,2017,2017-09-30
688,9614771539,"Octoer, 2, 17 CPT Code: 90801 - Psychiatric Diagnosis Interview",oct 2 17 cpt code: 90801 - psychiatric diagnosis terview,1,2,10,2017,2017-10-02
320,5014000276,Novmber 19 17 CPT Code: 90792: With medical services,nov 19 17 cpt code: 90792: with medical services,1,19,11,2017,2017-11-19
710,9880694088,"20, November 17 Total time of visit (in minutes):",20 nov 17 total time of visit ( m utes):,0,20,11,2017,2017-11-20


## Save to CSV

In [312]:
df.to_csv(ROOT+ "output/results.csv", index=True)